## NOTE: 
### To run this code, your Jupyter-notebook must be using Python3.8 or newer releases

# Prerequisites for calculation

Before I begin solving the tasks presented in the exercises I will define a set of constants to be used throughout this document. The meaning of the individual constants are briefly described in the appropriate comments.

In [1]:
import numpy as np

k_b = 1.38e-23                  #The boltzmann constant
T = 293                         #Example temperature (in Kelvin)
beta = 1 / (k_b * T)            #The inverse-temperature-constant for the example-temp

box_k = 600*k_b*T               #Spring-constant for box-walls

hardcore_diameter = 1e-3        #Particle-diameter for hardcore interaction
hardcore_pot = np.inf           #Particle-potential for hardcore interaction

N = 1000                        #Example amount of particles

epsilon = 1                     #Lennard-jones potential-constant
a = 0.1                         #Lennard-jones distance-constant


# Abstract box-system definitions
In order to quickly switch between using various inter-particle-potentials along with box-potentials for the particles in the $L\times L$ box, I will define a stateFunctions-class. Instances of this class will contain information about the potentials to be used, along with appropriate functions-definitions of these.  
  
This section is split into two code-boxes, where the first one was written as the .pyi file corresponding to the second one. The first code-block gives an overview of all functions defined in the second one, but is strictly speaking not neccesary.  
  
In this section I implement functions that calculate the energy and pressure for a given microstate. Each particle that lies outside the box would classically be influenced as if it were in a spring-system with spring-constant $k=box_k$. In a single dimension, the pressure-contribution from a single particle a distance $d$ outside the box would be calculated as $k\cdot d / A$, where $A$ is some relevant area. In our scenario, it is not neccesarily easy to define how the pressure should be calculated, especially outside the box-corners. I have chosen to sum up pressure-contribution from the x-direction and y-direction as if they were independent, and chosen $A=4\cdot L$, the one-dimensional area of the box. This allows calculating some pressure for a given microstate (though in reality pressure is a macroscopic quantity).  
  
Furthermore, the following section implements a series of energy-methods. The inter-particle energy-methods are Lennard-Jones, Hardcore, and No-interaction, all of which are coupled with the box-spring-system-energy as defined in the exercise-text. The energy-methods take in a system-state (a set of particle-positions) and yield a floating-point energy-value.  
  
All methods are compiled with jit to allow for fast execution.

In [2]:
from __future__ import annotations
from typing import List, Tuple, Callable, Dict, NamedTuple
from constants import hardcore_pot, hardcore_diameter, a, epsilon
from numba import njit
import numpy as np
from enum import Enum


class ENERGY(Enum):
    HARDCORE_ENERGY         :int = 1
    BOX_ENERGY              :int = 2
    HARDCORE_AND_BOX_ENERGY :int = 3
    LJ_AND_BOX_ENERGY       :int = 4


class stateFunctions:
    pressure: Callable[[np.ndarray], float]
    hardcore_energy: Callable[[np.ndarray], float]
    box_energy: Callable[[np.ndarray], float]
    box_and_hardcore_energy: Callable[[np.ndarray], float]
    box_and_lj_energy: Callable[[np.ndarray], float]
    single_particle_pressure: Callable[[np.ndarray, int], float]
    single_particle_box_energy: Callable[[np.ndarray, int], float]
    single_particle_hardcore_energy: Callable[[np.ndarray, int], float]
    single_particle_box_and_hardcore_energy: Callable[[np.ndarray, int], float]
    single_particle_box_and_lj_energy: Callable[[np.ndarray, int], float]

    def __init__(self, L: float, box_k: float, **kwargs):
        """
        Creates state-functions object with parameter L and box_k
        :param L: float
        :param box_k: float
        """
        ...

    def compile(self):
        """
        Recompile the energy- and pressure- methods of stateFunctions
        :return: None
        """
        ...

    def set_energy_type(self, energy_type: ENERGY):
        ...

    def get_energy(self) -> Callable[[np.ndarray], float]:
        ...

    def get_single_particle_energy(self) -> Callable[[np.ndarray, int], float]:
        ...

    def get_pressure(self) -> Callable[[np.ndarray], float]:
        ...

    def get_single_particle_pressure(self) -> Callable[[np.ndarray, int], float]:
        ...


In [3]:

class stateFunctions:
    pressure: Callable[[np.ndarray], float]
    hardcore_energy: Callable[[np.ndarray], float]
    box_energy: Callable[[np.ndarray], float]
    box_and_hardcore_energy: Callable[[np.ndarray], float]
    box_and_lj_energy: Callable[[np.ndarray], float]
    single_particle_pressure: Callable[[np.ndarray, int], float]
    single_particle_box_energy: Callable[[np.ndarray, int], float]
    single_particle_hardcore_energy: Callable[[np.ndarray, int], float]
    single_particle_box_and_hardcore_energy: Callable[[np.ndarray, int], float]
    single_particle_box_and_lj_energy: Callable[[np.ndarray, int], float]

    def __init__(self, L: float, box_k: float, **kwargs):
        self.L = L
        self.box_k = box_k
        self.compile()
        self.energy_type = kwargs.get("energy_type", ENERGY.HARDCORE_AND_BOX_ENERGY)

    def set_energy_type(self, energy_type: ENERGY):
        self.energy_type = energy_type

    def get_energy(self) -> Callable[[np.ndarray], float]:
        return {ENERGY.HARDCORE_ENERGY: self.hardcore_energy,
                ENERGY.BOX_ENERGY: self.box_energy,
                ENERGY.HARDCORE_AND_BOX_ENERGY: self.box_and_hardcore_energy,
                ENERGY.LJ_AND_BOX_ENERGY: self.box_and_lj_energy}[self.energy_type]

    def get_pressure(self) -> Callable[[np.ndarray], float]:
        return self.pressure

    def get_single_particle_energy(self) -> Callable[[np.ndarray, int], float]:
        return {ENERGY.HARDCORE_ENERGY: self.single_particle_hardcore_energy,
                ENERGY.BOX_ENERGY: self.single_particle_box_energy,
                ENERGY.HARDCORE_AND_BOX_ENERGY: self.single_particle_box_and_hardcore_energy,
                ENERGY.LJ_AND_BOX_ENERGY: self.single_particle_box_and_lj_energy}[self.energy_type]

    def get_single_particle_pressure(self) -> Callable[[np.ndarray, int], float]:
        return self.single_particle_pressure

    def compile(self):
        L = self.L
        box_k = self.box_k

        @njit()
        def pressure(state: np.ndarray) -> float:
            pressure_contrib = np.sum(np.where(state < 0, -state, 0)) + \
                               np.sum(np.where(state > L, state - L, 0))
            pressure_contrib *= box_k / L / 4
            return pressure_contrib

        @njit()
        def single_particle_pressure(state:np.ndarray, i: int):
            return pressure(state[i])

        @njit()
        def hardcore_energy(state: np.ndarray) -> float:
            particle_amt = len(state)
            energy_contrib = 0.0
            for i in range(particle_amt):
                for j in range(i+1, particle_amt):
                    energy_contrib += hardcore_pot if np.linalg.norm(state[i] - state[j]) < hardcore_diameter else 0
            return energy_contrib

        @njit()
        def LJ_energy(state: np.ndarray) -> float:
            particle_amt = len(state)
            energy_contrib = 0.0
            for i in range(particle_amt):
                for j in range(i+1, particle_amt):
                    r = np.linalg.norm(state[i]-state[j])
                    energy_contrib += epsilon*((a/r)**12 - 2*(a/r)**6)
            return energy_contrib

        @njit()
        def single_particle_LJ_energy(state: np.ndarray, i: int) -> float:
            energy_contrib = 0.0
            particle_i = state[i]
            for j in range(len(state)):
                if j != i:
                    r = np.linalg.norm(particle_i - state[j])
                    energy_contrib += epsilon*((a/r)**12 - 2*(a/r)**6)
            return energy_contrib

        @njit()
        def single_particle_hardcore_energy(state: np.ndarray, i: int) -> float:
            energy_contrib = 0.0
            particle_i = state[i]
            for j in range(len(state)):
                if j != i:
                    energy_contrib += hardcore_pot if np.linalg.norm(particle_i - state[j]) < hardcore_diameter else 0
            return energy_contrib


        @njit()
        def box_energy(state: np.ndarray) -> float:
            energy_contrib: float = 1 / 2 * box_k * np.sum(np.where(state < 0, state ** 2, 0)) \
                                    + 1 / 2 * box_k * np.sum(np.where(state > L, (state - L) ** 2, 0))
            return energy_contrib

        @njit()
        def single_particle_box_energy(state: np.ndarray, i: int) -> float:
            return box_energy(state[i])

        @njit()
        def box_and_hardcore_energy(state: np.ndarray) -> float:
            return box_energy(state) + hardcore_energy(state)

        @njit()
        def single_particle_box_and_hardcore_energy(state: np.ndarray, i: int) -> float:
            return single_particle_box_energy(state, i) + single_particle_hardcore_energy(state, i)

        @njit()
        def box_and_LJ_energy(state: np.ndarray):
            return box_energy(state) + LJ_energy(state)

        @njit()
        def single_particle_box_and_LJ_energy(state: np.ndarray, i: int):
            return single_particle_box_energy(state, i) + single_particle_LJ_energy(state, i)

        self.pressure = pressure
        self.hardcore_energy = hardcore_energy
        self.box_energy = box_energy
        self.box_and_hardcore_energy = box_and_hardcore_energy
        self.box_and_lj_energy = box_and_LJ_energy
        self.single_particle_pressure = single_particle_pressure
        self.single_particle_box_energy = single_particle_box_energy
        self.single_particle_hardcore_energy = single_particle_hardcore_energy
        self.single_particle_box_and_hardcore_energy = single_particle_box_and_hardcore_energy
        self.single_particle_box_and_lj_energy = single_particle_box_and_LJ_energy






Now I will define the actual systems used in simulations. These 'systems' consist of a state, describing the positions of all its particles, along with a series of parameters specifying the behaviour of the system. One such parameter is the 'state-functions'-object the system will use to calculate energies for a given state. Furthermore, the system has a function named 'explore' which will propagate the system-state with the metropolis Monte-Carlo method in accordance with its parameters. Like before, I have split the definitions into a descriptive '.pyi'-code-block, and another code block that actually implements all the methods. 

In [4]:
'''
class System:
    state: np.ndarray
    particle_amt: int
    dimension: int
    L: float
    jump_scale: float

    pressure: Callable[[np.ndarray], float]
    energy: Callable[[np.ndarray], float]

    def __init__(self, particle_amt: int, L: float, state_functions: stateFunctions, **kwargs): ...

    def explore(self, iterations: int, log_interval: int = 1):
        """
        Jitted method of performing multiple goto_next-steps, moving a single, randomly selected
        particle at a time
        :param iterations: int; iterations to perform
        :return:
        """
        ...'''

'\nclass System:\n    state: np.ndarray\n    particle_amt: int\n    dimension: int\n    L: float\n    jump_scale: float\n\n    pressure: Callable[[np.ndarray], float]\n    energy: Callable[[np.ndarray], float]\n\n    def __init__(self, particle_amt: int, L: float, state_functions: stateFunctions, **kwargs): ...\n\n    def explore(self, iterations: int, log_interval: int = 1):\n        """\n        Jitted method of performing multiple goto_next-steps, moving a single, randomly selected\n        particle at a time\n        :param iterations: int; iterations to perform\n        :return:\n        """\n        ...'

In [5]:

class System:
    state: np.ndarray
    particle_amt: int
    dimension: int
    L: float
    jump_scale: float

    pressure: Callable[[np.ndarray], float]
    energy: Callable[[np.ndarray], float]

    def __init__(self, particle_amt: int, L: float, state_functions: stateFunctions, **kwargs):
        self.particle_amt = particle_amt
        self.positional_dimension = 2           #2D system
        self.dimension = self.particle_amt*self.positional_dimension
        self.L = L
        self.state_functions = state_functions
        self.jump_scale = kwargs.get("jump_scale", 1)

        self.compile()

        #Generating initial random state
        self.state = np.random.random((self.particle_amt, self.positional_dimension)) * L

    def compile(self):
        jump_scale = self.jump_scale
        energy = self.state_functions.get_energy()
        pressure = self.state_functions.get_pressure()
        single_energy = self.state_functions.get_single_particle_energy()
        single_pressure = self.state_functions.get_single_particle_pressure()

        pos_dim = self.positional_dimension


        @njit()
        def jitted_explore(initial_state: np.ndarray, iterations: int, log_interval=100) -> Tuple[np.ndarray, List[np.ndarray]]:
            def particle_choice(state):
                i = np.random.randint(0, len(state))
                return i

            def jump():
                stepped_length = np.random.uniform(-jump_scale, jump_scale, (pos_dim,))
                return stepped_length

            def goto_next(state, i):
                single_step = jump()
                current_energy = single_energy(state, i)
                position_i_current = state[i] + 0
                state[i] += single_step
                next_energy = single_energy(state, i)
                delta_energy = next_energy - current_energy
                if not (delta_energy < 0 or np.random.random() < np.exp(-beta * delta_energy)) :
                    state[i] = position_i_current
                return state

            pressures = []

            state = initial_state
            for iter in range(iterations):
                i = particle_choice(state)
                state = goto_next(state, i)
                if iter % log_interval == 0:
                    pressures.append(pressure(state))

            return state, [pressures]

        self._explore = jitted_explore


    def __len__(self):
        return self.particle_amt

    def __getitem__(self, index: int):
        return self.state[index]

    def explore(self, iterations: int, log_interval: int = 1):
        state, values = self._explore(self.state, iterations, log_interval)
        self.state = state
        return values


All the exercises may now be solved in a short and concise way. 
  
### Task a)
For no particular reason I have chosen box-lengths $L$ in the interval $[0.1\text{m}, 10\text{m}]$ for testing the validity of the Ideal-Gas Law in an $L\times L$ box with the Metropolis Monte Carlo method.  
  
The jump-scale variable (briefly mentioned in the exercise-text, although it differs from the one defined in code by a factor 2) is here set to $0.08$, meaning that a particle may be moved at most $0.08\text{m}$ during Metropolis-Monte-Carlo iteration. There is no particular reason for choosing $0.08\text{m}$, other than that it seems to allow for fairly fast propagation of the particles, at the same time as it doen't reject too many proposed steps (as the particles rarely end up far outside the box, where the Monte-Carlo method most definitely would reject propagation and the state would remain unchanged). Choosing step-length $0.08\text{m}$ generates a fair amount of different microstates.  
  
Before I actually begin calculating pressures in the Monte-Carlo method, I will perform $500000$ iterations to allow the system to 'stabilize' (stabilize in a macroscopic sense). The system initiates by spreading the particles uniformly in the $L\times L$ box, but one needs to allow a few particles to venture outside the box before one begins calculating pressures and energies. Otherwise, these values would likely be unrepresentatively low.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.lines import Line2D

lower_l_lim = 0.1    #10^-1 = min_l
upper_l_lim = 1      #10^1 = max_l

lengths_amount = 20

particle_amt = 200

stabilization_iterations = 100000  #Iterations used for allowing the system to stabilize
averaging_iterations = 1000000      #Iterations used for calculating an average

fig, ax = plt.subplots(1,1)
lengths = np.logspace(lower_l_lim, upper_l_lim, lengths_amount)
pressures = []
print("^"*lengths_amount)
for length in lengths:
    state_fcns = stateFunctions(length, box_k, energy_type = ENERGY.BOX_ENERGY)
    sys = System(particle_amt, length, state_fcns, jump_scale = 0.08)
    sys.explore(stabilization_iterations, log_interval=1000)
    
    pressure_values = sys.explore(averaging_iterations, log_interval=100) #Pressure is only logged every 1000 iterations 
                                                        #as it is unneccesary to calculate pressures for similar microstates
    average_pressure = np.average(pressure_values)
    pressures.append(average_pressure)
    print("*", end="")
ax.plot(lengths, pressures, label="simulation p", color="r")
ideal_pressures = particle_amt * k_b * T / (lengths**2)

ax.plot(lengths, ideal_pressures, label= "ideal p", color = "b", linestyle="--")
ax.set_yscale("log")
ax.set_xscale("log")
ax.legend()
ax.set_xlabel("length [m]")
ax.set_ylabel("pressure [pa]")

fig.show()

^^^^^^^^^^^^^^^^^^^^
******

There seems to be good correspondence between our calculations and the ideal-gas-law. In order to demonstrate that the particles in fact move around (not as described by the laws of motion, but generating a set of microstates), I will construct a gif showing the particle-trails during the monte-carlo-iterations. In this case I will only render the simulation with $20$ particles, as including more would only clutter the plot.

In [ ]:
from matplotlib.animation import FuncAnimation
plot_amt = 20      #antall partikler

anim_iters = 20   #Iterasjoner mellom hvert plot
anim_frames = 300   #Antall frames

L = 1

if __name__ == "__main__":

    #Plotting a single system

    state_fcns = stateFunctions(L, box_k, energy_type=ENERGY.BOX_ENERGY)
    sys = System(plot_amt, L, state_fcns, jump_scale=0.08)
    fig, ax = plt.subplots(1,1, figsize=(16,16))
    data_storage = [elem for x_data, y_data in sys.state for elem in ([x_data], [y_data])]   #yeehaw
    curves:List[Line2D] = ax.plot(*data_storage)
    ax.set_aspect("equal")
    ax.set_xlim([-0.2, 1.2])
    ax.set_ylim([-0.2, 1.2])
    ax.axvline(1, color="k"); ax.axvline(0, color="k")
    ax.axhline(0, color="k"), ax.axhline(1, color="k")


    def animate(*args):
        pressure, = sys.explore(anim_iters, log_interval=anim_iters)
        print("*", end="")
        ax.set_title(f"p={pressure[0]:.3g} pa")
        for enum, (x_data, y_data) in enumerate(sys.state):
            data_storage[2*enum].append(x_data)
            data_storage[2*enum+1].append(y_data)
            data_storage[2*enum] = data_storage[2*enum][-10:]
            data_storage[2*enum+1] = data_storage[2*enum+1][-10:]
        for enum, line in enumerate(curves):
            line.set_data((data_storage[2*enum], data_storage[2*enum + 1]))
        return curves
    anim = FuncAnimation(fig, animate, anim_frames)
    anim.save("anim.gif", fps=30)
    print("")
    print("anim.gif may now be found in CWD")
    del fig, ax, anim
    


In [ ]:
from IPython.display import Image
Image(url="anim.gif")

It is evident that the particles move around sufficiently to generate several microstates.

## Task b)
The methods used here are largely the same as in $\textbf{(a)}$

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.lines import Line2D

particle_amt = 200

stabilization_iterations = 10000  #Iterations used for allowing the system to stabilize
averaging_iterations = 100000      #Iterations used for calculating an average

fig, ax = plt.subplots(1,1)
lengths = np.logspace(lower_l_lim, upper_l_lim, lengths_amount)
pressures = []
print("^"*lengths_amount)
for length in lengths:
    state_fcns = stateFunctions(length, box_k, energy_type = ENERGY.HARDCORE_AND_BOX_ENERGY)
    sys = System(particle_amt, length, state_fcns, jump_scale = 0.08)
    sys.explore(stabilization_iterations, log_interval=1000)
    
    pressure_values = sys.explore(averaging_iterations, log_interval=100) #Pressure is only logged every 1000 iterations 
                                                        #as it is unneccesary to calculate pressures for similar microstates
    average_pressure = np.average(pressure_values)
    pressures.append(average_pressure)
    print("*", end="")
ax.plot(lengths, pressures, label="simulation p", color="r")
ideal_pressures = particle_amt * k_b * T / (lengths**2-np.pi*hardcore_diameter**2/2)

ax.plot(lengths, ideal_pressures, label= "ideal p", color = "b", linestyle="--")
ax.set_yscale("log")
ax.set_xscale("log")
ax.legend()
ax.set_xlabel("length [m]")
ax.set_ylabel("pressure [pa]")

fig.show()